In [0]:
import os, sys, shutil, h5py, json
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import scipy
import numpy as np
import tensorflow as tf
import pickle
import numpy
import matplotlib.pyplot as plt
import glob

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from shutil import copy

from keras.models import Model
from keras.models import model_from_json
from keras.models import Model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Cropping2D, ZeroPadding2D, Permute, Reshape, Input, Flatten
from keras.layers.normalization import BatchNormalization
import keras.optimizers
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

%cd /content/gdrive/My\ Drive/Deep_Learning/Images_Cropped/

with open("Cropped_Pickle_OGs.pickle", 'rb') as handle:
    sample0 = pickle.load(handle)

with open("Cropped_Pickle_rotated_1.pickle", 'rb') as handle:
    sample1 = pickle.load(handle)

with open("Cropped_Pickle_rotated_2.pickle", 'rb') as handle:
    sample2 = pickle.load(handle)

with open("Cropped_Pickle_rotated_3.pickle", 'rb') as handle:
    sample3 = pickle.load(handle)

total_samples = np.r_[(sample0, sample1, sample2, sample3)]

# Free up RAM
del sample0
del sample1
del sample2
del sample3

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Deep_Learning/Images_Cropped


In [0]:
print(total_samples.shape)

# 83 without fragments
# 81 with fragments

healthy_samples = []
splinter_samples = []

for picture, label, augmentation, origin in total_samples:
  if (augmentation == "Original"):
    if label == 0:
      healthy_samples.append([picture.reshape(420, 420, 1), label, augmentation, origin])
    elif label == 1:
      splinter_samples.append([picture.reshape(420, 420, 1), label, augmentation, origin])
    else:
      print("Some data was not labelled.")
  
healthy_samples = np.array(healthy_samples)
splinter_samples = np.array(splinter_samples)

print(healthy_samples.shape)
print(splinter_samples.shape)

(8528, 4)
(83, 4)
(81, 4)


In [0]:
healthy_origins = np.unique(healthy_samples[:, 3])
splinter_origins = np.unique(splinter_samples[:, 3])

healthy_split = int(0.8*len(healthy_origins))
splinter_split = int(0.8*len(splinter_origins))

training_samples = []
test_samples = []

for picture, label, augmentation, origin in healthy_samples:
  if origin in healthy_origins[:healthy_split]: # 0-80%
    training_samples.append([picture, label, augmentation, origin])
  elif origin in healthy_origins[healthy_split:]: # 80% - 100%
    test_samples.append([picture, label, augmentation, origin])
  else:
    print("idk")

for picture, label, augmentation, origin in splinter_samples:
  if origin in splinter_origins[:splinter_split]: # 0% - 80%
    training_samples.append([picture, label, augmentation, origin])
  elif origin in splinter_origins[splinter_split:]: # 80% - 100%
    test_samples.append([picture, label, augmentation, origin])
  else:
    print("idk")

training_samples = np.array(training_samples)
test_samples = np.array(test_samples)

print(training_samples.shape)
print(test_samples.shape)

(130, 4)
(34, 4)


In [0]:
'''
# Save the training and testing set, to avoid having to load the 8400~ samples in the future
with open('Training_set_NoAugmentation_NoNormalization.pickle', 'wb') as handle:
    pickle.dump(training_samples, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Test_set_NoAugmentation_NoNormalization.pickle', 'wb') as handle:
    pickle.dump(test_samples, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

In [0]:
np.random.seed(45)
np.random.shuffle(training_samples)
np.random.shuffle(test_samples)

training_images = []
training_labels = []
test_images = []
test_labels = []

for picture, label, _, _ in training_samples:
  training_images.append(picture)
  training_labels.append(label)

for picture, label, _, _ in test_samples:
  test_images.append(picture)
  test_labels.append(label)

training_images = np.array(training_images)
training_labels = np.array(training_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Clear up RAM
del training_samples


In [0]:
# Intialize model
def initialize_conv_network(input_, target_):
    input_img = Input(shape=input_.shape[1:])
    x = input_img

    # ---------
    x = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(x)
    x = Conv2D(32, (5, 5), strides=(2, 2), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(32, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)
    x = Dropout(0.3)(x)

    # ---------
    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)
    x = Dropout(0.3)(x)

    # ---------
    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)
    x = Dropout(0.3)(x)

    # ---------
    # No padding here
    x = Conv2D(512, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = Conv2D(512, (1, 1), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    # ---------
    # No padding here
    x = Conv2D(15, (1, 1), strides=(1, 1), kernel_initializer='he_uniform', padding='valid', data_format='channels_last')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = Flatten()(x)

    x = Dense(2, kernel_initializer='he_normal')(x)
    final_output = Activation('softmax')(x)
    model = Model(inputs=input_img, outputs=final_output)
    #print(model.summary())
    #exit()
    return model

In [0]:
model = initialize_conv_network(training_images, training_labels)
print(model.summary())













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 420, 420, 1)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 424, 424, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 210, 210, 32)      832       
_________________________________________________________________
batch_normalization_1 (Batch (None, 210, 210, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 210, 210, 32)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 212, 212, 32)      0       

In [0]:
PATIENCE = 50
#VAL_SPLIT = 0.25
BATCH_SIZE = 2
EPOCHS = 100

save_name = 'aNoAugmentation'
loss_func = 'sparse_categorical_crossentropy'
AO = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

max_train_acc = []
max_val_acc = []

for i in range(10):
  print(i)
  if i == 0:
    model.compile(loss=loss_func, optimizer=AO, metrics=['accuracy'])
    json_string = model.to_json()
    with open(str(save_name) + '.json', 'w') as f:
        f.write(json_string)

  early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=PATIENCE, verbose=1, mode='max')
  checkpoint = ModelCheckpoint(str(save_name) + "_" + str(i) +  '.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
  callbacks_list = [checkpoint, early_stopping]

  history = model.fit(training_images, training_labels, batch_size=BATCH_SIZE, validation_data=(test_images, test_labels), epochs=EPOCHS, callbacks=callbacks_list)
  scipy.io.savemat(str(save_name) + "_" + str(i) + '_TrainingLog.mat', {"val_acc": history.history['val_acc'],
                                                            "acc": history.history['acc'],
                                                            "loss": history.history['loss']})
  
  print("Max train acc: " + str(np.max(history.history['acc'])))
  print("Max val acc: " + str(np.max(history.history['val_acc'])))
  max_train_acc.append(np.max(history.history['acc']))
  max_val_acc.append(np.max(history.history['val_acc']))

  model = initialize_conv_network(training_images, training_labels) # Restarts weights
  model.compile(loss=loss_func, optimizer=AO, metrics=['accuracy'])
  

0


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 130 samples, validate on 34 samples
Epoch 1/100
130/130 [==============================] - 11s 85ms/step - loss: 1.0909 - acc: 0.4615 - val_loss: 0.6660 - val_acc: 0.5588

Epoch 00001: val_acc improved from -inf to 0.55882, saving model to aNoAugmentation_0.hdf5
Epoch 2/100
130/130 [==============================] - 2s 12ms/step - loss: 0.9367 - acc: 0.5154 - val_loss: 0.6234 - val_acc: 0.7941

Epoch 00002: val_acc improved from 0.55882 to 0.79412, saving model to aNoAugmentation_0.hdf5
Epoch 3/100
130/130 [==============================] - 2s 13ms/step - loss: 0.9374 - acc: 0.5769 - val_loss: 0.8757 - val_acc: 0.4706

Epoch 00003: val_acc did not improve from 0.79412
Epoch 4/100
130/130 [==============================] - 2s 13ms/step - loss: 0.8666 - acc: 0.5154 - val_loss: 0.6837 - val_acc: 0.6765

Epoch 00004: val_acc did not improve from 0.79412
Epoch 5/100
130/130 [========

In [0]:
print(max_train_acc)
print(max_val_acc)

from statistics import stdev
print(np.mean(max_train_acc))
print(stdev(max_train_acc))

print(np.mean(max_val_acc))
print(stdev(max_val_acc))


[1.0, 1.0, 1.0, 0.9923076923076923, 0.9384615384615385, 0.9769230769230769, 1.0, 0.9076923076923077, 0.9923076923076923, 1.0]
[0.7941176470588235, 0.6764705882352942, 0.7647058823529411, 0.8235294117647058, 0.7647058823529411, 0.7647058823529411, 0.6470588235294118, 0.7352941176470589, 0.7647058823529411, 0.7352941176470589]
0.9807692307692306
0.03207691282870975
0.7470588235294118
0.05224671605676164
